<a href="https://colab.research.google.com/github/Just-Aymz/Student-Database-Management/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import Libraries**
___

In [1]:
import pandas as pd
import numpy as np

import matplotlib as plt
import seaborn as sns

from sklearn.impute import KNNImputer
from scipy.stats import kurtosis, skew, shapiro
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from scipy import stats

from typing import Optional

## **Read Files**
___


In [2]:
# Set Pandas to display all columns
pd.set_option('display.max_columns', None)

# Store the Github repository and file name as a variable
url = "https://raw.githubusercontent.com/Just-Aymz/Diabetes/refs/heads/main/diabetes.csv?token=GHSAT0AAAAAAC5FR3CDXXSUBWI5R22NMR6OZ4CV2LA"

# Store the dataframe as a variable
df = pd.read_csv(url)

## **Dataset Identification**
___

In [3]:
# Return the shape and the size of the dataset.
print(f"The shape of the dataset is: {df.shape}\nThe size of the dataset is: {df.size}")

The shape of the dataset is: (768, 9)
The size of the dataset is: 6912


In [4]:
# Return the axes of the dataset
df.axes

[RangeIndex(start=0, stop=768, step=1),
 Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
        'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
       dtype='object')]

In [5]:
# Return the summary of each feature in the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [6]:
# Return a sample of the top 5 rows of the dataset
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
# Return a samole of the bottom 5 rows of the dataset
df.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


# **Data Preprocessing**
___


### 1. **Data Cleaning**
Dataset preprocessing is the process of cleaning and transforming raw data into a format suitable for analysis or machine learning. It involves handling missing values, removing duplicates, correcting inconsistencies, scaling or normalizing features, encoding categorical variables, and selecting relevant features. The goal is to ensure that the data is accurate, consistent, and structured to improve the performance of analytical models and algorithms.



#### 1.1 **Duplicate Values**
In the context of a dataframe, a record can be a duplicate of another record, if all the data values of each feature are identical. A record can also be considered a duplicate if the data values of a specified subset of features are identical. In this section, we look to identify and remove the duplicate values within the dataset.


In [8]:
dupes = df.duplicated().sum()
if dupes != 0:
  print(f"Total number of duplicate values: {dupes}")
else:
  print("There are no duplicate values in the dataset")

There are no duplicate values in the dataset


#### 1.2. **Null Values**
___

In [9]:
nulls = np.count_nonzero(df.isnull().values)
if nulls != 0:
  print(f"Total number of null values: {nulls}")
else:
  print(f"There are no duplicate values in the dataset")

There are no duplicate values in the dataset


#### 1.3 **Outlier Values**
___

In [10]:
def outlier_Count(column: str,
                  features_with_outliers: Optional[list]=None,
                  dataframe: Optional[pd.DataFrame]=df,
                  store: Optional[bool]=False):
  # Store the q1 and q3 values
  q1 = dataframe[column].quantile(0.25)
  q3 = dataframe[column].quantile(0.75)

  # Store the IQR value
  IQR = q3 - q1

  # Use the IQR to find the upper and the lower limits
  upper = dataframe[column] > q3 + (IQR * 1.5)
  lower = dataframe[column] < q1 - (IQR * 1.5)

  # Store the outlier values
  outliers = dataframe[column][upper | lower]

  if store == False:
    return len(outliers)
  if (store == True) & (len(outliers) > 0):
    features_with_outliers.append(column)

for feature in df.drop(columns=['Outcome']):
  if ((outliers := outlier_Count(feature)) > 0):
    print(f"{feature}:\nTotal Outliers: {outliers}\n")


Pregnancies:
Total Outliers: 4

Glucose:
Total Outliers: 5

BloodPressure:
Total Outliers: 45

SkinThickness:
Total Outliers: 1

Insulin:
Total Outliers: 34

BMI:
Total Outliers: 19

DiabetesPedigreeFunction:
Total Outliers: 29

Age:
Total Outliers: 9



#### 1.4 **Miscellaneous Errors**
___
These errors can be generalized as the errors found in the data values of the records in the dataframe. This step includes converting features to correct datatypes. In this section we look to correct those data errors in order to feed a clean dataset to our machine learning models.


##### 1.4.1. **Numeric Features**

In [11]:
# Return a summary of the descriptive statistics of the dataset.
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Based on the minimum values of the Glucose, BloodPressure, SkinThickness, Insulin and BMI, we can logically deduce that these are input errors, as these values are not physiologically possible. Therefore, we can try and replace these values with N/A values as it can be assumed that these values were missing from the data collection point.


In [12]:
# Store the features that will need to have the 0 values replaced with Nan
# values
error_features = [
    'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI'
]

for column in df:
  # If the feature of the dataframe is in the error_features list
  if column in error_features:
    # Replace the 0 values with nan.
    df[column] = df[column].replace(0, np.nan)

In [13]:
# Return the total number of null values in the dataframe
new_nulls = np.count_nonzero(df.isnull().values)
print(f"Total number of null values: {new_nulls}")

Total number of null values: 652


In [14]:
# Return the total number of null values per feature of the dataset
df.isnull().sum()

,0
Pregnancies,0
Glucose,5
BloodPressure,35
SkinThickness,227
Insulin,374
BMI,11
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [15]:
def statistics(column: str,
               dataframe: Optional[pd.DataFrame]=df,
               dct: Optional[bool]=False) -> None:
    """
    A function that outputs statistics for features with more than 12 unique
    values.The skew and Kurtosis values, along with the p-value from the
    Shapiro-Wilk test. These values help evaluate the distribution of the
    features of the dataframe passed in to the dataframe parameter.

    Args:
        dataframe:
            - A pandas dataframe object.

    Returns:
        None
    """
    skew_and_distribution = {}
    _skew = skew(dataframe[column])
    _kurtosis = kurtosis(dataframe[column], fisher=True)
    print(
        f"\n{column}\n"
        f"skew: {_skew:.4f}\n"
        f"kurtosis: {_kurtosis:.4f}"
    )

    # Perform Shapiro-Wilk test
    stat, p_value = shapiro(dataframe[column])
    skew_and_distribution.update({'test_score': f'{stat:.4f}',
                                  'p_value': f'{p_value:.4f}',
                                  'feature': column})

    # Print the results
    print(f"Shapiro-Wilk Normality test: {stat:.4f}")
    print(f"P-value: {p_value:.4f}")

    # Interpret the p-value
    alpha = 0.05
    if p_value > alpha:
        print("The data is likely normally distributed (fail to reject H0).")
        skew_and_distribution.update(
            {'hypothesis_result': 'fail to reject H0'}
        )
    else:
        print("The data is not normally distributed (reject H0).")
        skew_and_distribution.update({'hypothesis_result': 'reject H0'})

    # Check the absolute values of each skew value of a feature
    if np.abs(_skew) < 0.5:
        print('distribution is almost symmetrical')
        skew_and_distribution.update({'skew': 'almost symmetrical'})
    elif 0.5 <= np.abs(_skew) <= 1:
        print('distribution is modertely skewed')
        skew_and_distribution.update({'skew': 'modertely skewed'})
    else:
        print('distribution is highly skewed')
        skew_and_distribution.update({'skew': 'highly skewed'})

    # Extremity of tail distribution
    if _kurtosis > 0:
        print(
            f'Leptokurtic distribution - heavier tails and a sharper '
            f'peak than the normal distribution.\n'
            f'This type of distribution is often associated with higher '
            f'peakedness and a greater probability of extreme values.\n'
        )
    elif _kurtosis < 0:
        print(
            f'Platykurtic distribution - lighter tails and a flatter '
            f'peak than the normal distribution.\n'
            f'This type of distribution is often associated with less  '
            f'peakedness and a lower probability of extreme values.\n'
        )
    else:
        print(
            f'Mesokurtic distribution - similar peak and tail shape as the  '
            f'normal distribution.\n'
            )

    # Return dictionary
    if dct == True:
        return skew_and_distribution

# Store a dataset with no null values to not affect the SHapiro-Wilk Normality
# Test.
non_null_df = df.dropna()
skew_and_distribution = []
for column in non_null_df.drop(columns=['Outcome']):
  feature_stats = statistics(column, non_null_df, dct=True)
  skew_and_distribution.append(feature_stats)


Pregnancies
skew: 1.3305
kurtosis: 1.4522
Shapiro-Wilk Normality test: 0.8530
P-value: 0.0000
The data is not normally distributed (reject H0).
distribution is highly skewed
Leptokurtic distribution - heavier tails and a sharper peak than the normal distribution.
This type of distribution is often associated with higher peakedness and a greater probability of extreme values.


Glucose
skew: 0.5159
kurtosis: -0.4924
Shapiro-Wilk Normality test: 0.9642
P-value: 0.0000
The data is not normally distributed (reject H0).
distribution is modertely skewed
Platykurtic distribution - lighter tails and a flatter peak than the normal distribution.
This type of distribution is often associated with less  peakedness and a lower probability of extreme values.


BloodPressure
skew: -0.0872
kurtosis: 0.7700
Shapiro-Wilk Normality test: 0.9899
P-value: 0.0087
The data is not normally distributed (reject H0).
distribution is almost symmetrical
Leptokurtic distribution - heavier tails and a sharper peak 

In [16]:
skew_dist_df = pd.DataFrame(skew_and_distribution)
skew_dist_df

,test_score,p_value,feature,hypothesis_result,skew
0,0.8530,0.0000,Pregnancies,reject H0,highly skewed
1,0.9642,0.0000,Glucose,reject H0,modertely skewed
2,0.9899,0.0087,BloodPressure,reject H0,almost symmetrical
3,0.9876,0.0020,SkinThickness,reject H0,almost symmetrical
4,0.8040,0.0000,Insulin,reject H0,highly skewed
5,0.9738,0.0000,BMI,reject H0,modertely skewed
6,0.8486,0.0000,DiabetesPedigreeFunction,reject H0,highly skewed
7,0.8383,0.0000,Age,reject H0,highly skewed


The KNNImputer (K-Nearest Neighbors Imputer) is an imputation method that estimates missing values by considering the k nearest neighbors in the feature space. It imputes missing values based on the values of these closest data points.

Given that the features with missing values contain outliers and do not follow a normal distribution (as indicated by the rejection of the null hypothesis in the Shapiro-Wilk Normality Test), KNNImputer is an appropriate choice to impute the missing data. This method does not assume a specific data distribution, making it more robust to outliers and non-normal data. Moreover, since the dataset is not too large, the computational cost of KNNImputer, although memory-intensive, is manageable

In [17]:
# Instantiate a KNNImputer object
imputer = KNNImputer()
imputed_df = pd.DataFrame(imputer.fit_transform(df))

# Map the column names to match the original dataframe
imputed_columns = list(range(9))
original_columns = df.columns
imputed_df.rename(
    dict(zip(imputed_columns, original_columns)), axis=1, inplace=True
)

### 2. **Data Transformation**
Data transformation refers to the process of converting data from one format or structure into another. This step is important in preparing your data for analysis, machine learning models, or other types of data processing. The main goal of data transformation is to make the dataset more suitable for analysis or machine learning by improving its structure, scaling, and features.


#### 2.1 **Feature Transformation and Scaling**


In [18]:
imputed_df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,121.598177,72.371354,29.114583,153.180208,32.427344,0.471876,33.240885,0.348958
std,3.369578,30.496543,12.206417,9.424100,98.385308,6.886588,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,23.000000,87.900000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,133.700000,32.090000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,35.000000,190.150000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [19]:
# Store the features with outliers
outlier_features = []
for feature in imputed_df.drop(columns=['Outcome']):
  column = outlier_Count(feature, outlier_features, imputed_df, store=True)

# Store the positive skewed features
positive_skew = (
    skew_dist_df[skew_dist_df['skew'] == 'highly skewed']['feature'].values
                                                                    .tolist()
)

# Store the moderately skewed features
moderate_skew = (
    list(set(outlier_features).difference(set(positive_skew))) + ['Glucose']
)

In [20]:
def feature_transformation(feature: str,
                           dataframe: Optional[pd.DataFrame]=imputed_df,
                           skew_columns: Optional[list]=positive_skew,
                           moderate_columns: Optional[list]=moderate_skew):

  if feature in skew_columns:
    if feature == 'Pregnancies':
      # Shift the feature records by 1 for the pregnancy features
      dataframe[feature] =dataframe[feature] + 1

    # Apply box-cox transformation to transform data and make it more normally
    # distributed.
    dataframe[feature], lambda_optimal = stats.boxcox(dataframe[feature])

    # Instantiate RobustScaler object
    scaler = RobustScaler()

    # Fit and Transform the feature
    dataframe[feature] = scaler.fit_transform(dataframe[[feature]])

  elif feature in moderate_columns:
    if (feature == 'BMI') | (feature == 'Glucose'):
      # Instantiate MinMaxScaler object
      scaler = MinMaxScaler()

      # Fit and transform the feature
      dataframe[feature] = scaler.fit_transform(dataframe[[feature]])

    else:
      # Instantiate StandardScaler object
      scaler = StandardScaler()

      # Fit and transform the feature
      dataframe[feature] = scaler.fit_transform(dataframe[[feature]])

for feature in imputed_df:
  feature_transformation(feature)

In [21]:
# Confirm that the features have been scaled
imputed_df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.473523,0.670968,-0.030443,0.624914,0.309029,0.314928,0.543250,0.823159,1.0
1,-0.526477,0.264516,-0.522308,-0.012166,-1.022156,0.171779,-0.063340,0.129022,0.0
2,0.701530,0.896774,-0.686263,-0.351943,0.273806,0.104294,0.614014,0.187212,1.0
3,-0.526477,0.290323,-0.522308,-0.649247,-0.448805,0.202454,-0.878490,-0.776619,0.0
4,-0.993550,0.600000,-2.653723,0.624914,0.301095,0.509202,1.807443,0.241715,1.0
...,...,...,...,...,...,...,...,...,...
763,0.890831,0.367742,0.297467,2.005255,0.393672,0.300613,-0.851826,1.048838,0.0
764,-0.226183,0.503226,-0.194398,-0.224527,0.277042,0.380368,-0.097381,-0.149107,0.0
765,0.338487,0.496774,-0.030443,-0.649247,-0.227946,0.163599,-0.452365,0.066763,0.0
766,-0.526477,0.529032,-1.014173,0.646150,0.004864,0.243354,-0.069443,0.752406,1.0


In [25]:
# Confirm that the transformations have not resulted in null values being
# created
imputed_null_values = np.count_nonzero(imputed_df.isnull().values)
print('Total number of null values after transformations:', end=' ')
print(f'{imputed_null_values}')

Total number of null values after transformations: 0


In [23]:
# Store the cleaned datasets
imputed_df.to_csv()

',Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome\n0,0.4735225590274271,0.6709677419354838,-0.03044268939714405,0.6249140092771587,0.30902881648637137,0.3149284253578733,0.5432500232986767,0.8231585980903307,1.0\n1,-0.526477440972573,0.264516129032258,-0.5223077410873306,-0.012166467437254289,-1.0221561581879957,0.17177914110429454,-0.06333963744217111,0.1290220201541681,0.0\n2,0.701529622310065,0.896774193548387,-0.6862627583173928,-0.3519427216849411,0.2738055609596457,0.10429447852760743,0.6140138823005531,0.18721239549374558,1.0\n3,-0.526477440972573,0.2903225806451613,-0.5223077410873306,-0.6492469441516673,-0.44880543349931107,0.20245398773006146,-0.878489610327134,-0.7766188695462681,0.0\n4,-0.9935504559995367,0.6000000000000001,-2.653722965078139,0.6249140092771587,0.30109513759328516,0.5092024539877302,1.8074434785632032,0.24171494142665687,1.0\n5,0.3384872026333125,0.4645161290322581,0.13351232783291814,-0.9040791348374323,-0.3